# AIAp Miniproject 2

## Contents

- [Imports](#imports)

## Imports

In [ ]:
from keras import utils as keras_utils

## Dataset

### b) Citation and description

Citation: 
> Llamas, J. (2017). Architectural Heritage Elements Dataset: 64 (creative commons) revised (Version 1) [Dataset; .Jpg in .Zip]. Fundación CARTIF. https://correo.cartif.es/home/joslla@cartif.es/Briefcase/Architectural_Heritage_Elements_image_Dataset/Architectural_Heritage_Elements_Dataset_128%28creative_commons%29.zip

- **Title:** Architectural Heritage Elements Dataset
- **Subtitle:** 128 (creative commons) revised
- **Version:** 1
- **Publication Date:** 2017-02-20
- **Author:** Jose Llamas
- **Organization:** Fundación CARTIF
- **Source:** https://correo.cartif.es/home/joslla@cartif.es/Briefcase/Architectural_Heritage_Elements_image_Dataset/Architectural_Heritage_Elements_Dataset_128%28creative_commons%29.zip
- **Media:** .jpg in .zip
- **Download:** 2024-03-26

The raw data was extracted with Windows. We then added it to our repository on the Gitlab instance of OST.

This dataset consists of 10437 RGB 64x64 jpg images classified in 11 categories:
- Altar: 828 images
- Apse: 505 images
- Bell tower: 1057 images
- Column: 1914 images
- Dome (inner): 589 images
- Dome (outer): 1175 images
- Flying buttress: 405 images
- Gargoyle (and Chimera): 1562 images
- Portal: 307 images
- Stained glass: 998 images
- Vault: 1097 images

As "flying buttress" and "portal" do not satisfy the minimal samples requirement of 500, we dropped them. Our adjusted dataset therefore fulfils the requirements:
- RGB images of 9 classes
- Each has more than 500 samples and the total is 9720 images
- They have a resolution of 64x64 pixels

### c) Splitting into training and test data

In [ ]:

keras_utils.image_dataset_from_directory(directory="./data/raw/Architectual_Heritage_Elements_Dataset_128(creative_commons)", image_size=(128, 128))

train_val_imgs, test_images, train_val_lables, test_labels = model_selection.train_test_split()

## Exploratory data analysis

## Exploratory data analysis

## Architecture 1 (underfitting)

## Architecture 2 (overfitting)

## Architecture 2 (optimized)

## Quantification of the model performance

## Discussion